# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 1.2.1
- **Date :** 05/11/2023
- **Heure :** 13:31
- **Données :** `ready_Evaporation.csv`
- **Tri :** par défaut
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Omar/ready_Evaporation.csv", index_col = 1)
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,18.800000,38.4,0.0,13.1,50.000000,22.0,13.0,46.0,20.0,1010.4,...,2009,1,1,38,0.000000,5.890486,5.497787,-31.667778,116.015,5
2009-01-02,22.000000,39.1,0.0,13.2,52.000000,4.0,31.0,38.0,23.0,1008.3,...,2009,1,2,38,0.000000,2.356194,3.926991,-31.667778,116.015,5
2009-01-03,15.900000,35.6,0.0,12.7,43.473477,13.0,28.0,58.0,29.0,1012.1,...,2009,1,3,38,2.896869,9.424778,4.319690,-31.667778,116.015,5
2009-01-04,12.456393,37.0,0.0,13.3,43.000000,24.0,20.0,45.0,19.0,1016.6,...,2009,1,4,38,5.497787,5.497787,5.890486,-31.667778,116.015,5
2009-01-05,21.800000,38.6,0.0,13.2,59.000000,15.0,15.0,35.0,15.0,1014.1,...,2009,1,5,38,0.000000,5.890486,5.105088,-31.667778,116.015,5


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2646 entries, 2009-01-01 to 2017-06-25
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         2646 non-null   float64
 1   MaxTemp         2646 non-null   float64
 2   Rainfall        2646 non-null   float64
 3   Sunshine        2646 non-null   float64
 4   WindGustSpeed   2646 non-null   float64
 5   WindSpeed9am    2646 non-null   float64
 6   WindSpeed3pm    2646 non-null   float64
 7   Humidity9am     2646 non-null   float64
 8   Humidity3pm     2646 non-null   float64
 9   Pressure9am     2646 non-null   float64
 10  Pressure3pm     2646 non-null   float64
 11  Cloud9am        2646 non-null   float64
 12  Cloud3pm        2646 non-null   float64
 13  Temp9am         2646 non-null   float64
 14  Temp3pm         2646 non-null   float64
 15  RainToday       2646 non-null   int64  
 16  RainTomorrow    2646 non-null   int64  
 17  year            2646 no

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

La méthode TSF (_time series forest_) appartient à la famille des approches de classification des séries temporelles basées sur les **intervalles** (_interval-based approaches_).

Il s'agit d'une adaptation aux séries temporelles de l'algorithme _random forest classifier_ (RCF).

La TSF utilise un arbre de décision pour chaque intervalle, les arbres de décision agrégés constituant la forêt. Chaque arbre de décision est un modèle d'apprentissage automatique qui attribue ensuite une classe à son intervalle de données. Étant donné que les arbres de décision s'entraînent sur un intervalle différent de la série temporelle globale, ils peuvent ne pas produire la même classification, d'où la nécessité du vote d'ensemble à la fin du processus.

Des études expérimentales ont démontré que la TSF est capable de produire de meilleurs résultats que la KNN + DTW.

Références :
1. https://developer.ibm.com/learningpaths/get-started-time-series-classification-api/what-is-time-series-classification
2. https://towardsdatascience.com/a-brief-introduction-to-time-series-classification-algorithms-7b4284d31b97
3. https://arxiv.org/abs/1602.01711

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[16, 19],
         [ 0,  9],
         [22, 25],
         [14, 24],
         [11, 14]]),
  array([[ 7, 11],
         [14, 19],
         [10, 20],
         [13, 17],
         [22, 25]]),
  array([[18, 24],
         [15, 19],
         [20, 23],
         [14, 19],
         [14, 21]]),
  array([[ 5, 20],
         [19, 23],
         [12, 15],
         [15, 23],
         [ 0,  8]]),
  array([[ 4, 19],
         [16, 24],
         [ 8, 13],
         [ 6,  9],
         [17, 20]]),
  array([[11, 24],
         [20, 24],
         [13, 19],
         [ 3, 11],
         [11, 14]]),
  array([[ 5,  8],
         [ 0, 18],
         [ 2, 24],
         [13, 16],
         [18, 21]]),
  array([[ 2, 10],
         [15, 23],
         [ 8, 12],
         [13, 17],
         [20, 23]]),
  array([[11, 20],
         [16, 23],
         [ 2,  6],
         [ 2,  5],
         [ 8, 18]]),
  array([[ 4, 19],
         [11, 14],
         [22, 25],
         [12, 20],
         [1

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,333,13
1,32,63


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       346
           1       0.83      0.66      0.74        95

    accuracy                           0.90       441
   macro avg       0.87      0.81      0.84       441
weighted avg       0.89      0.90      0.89       441

